In [1]:
import re
import math
import pickle
import chromadb
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import numpy as np
from collections import Counter, defaultdict

# from embedding import LawEmbeddings

In [2]:
# ipynb 파일용
import sys
py_files_path = Path.cwd().parent / "py_files"
sys.path.append(str(py_files_path))

from embedding import LawEmbeddings

In [3]:
class NaiveSearchEngine():
    def __init__(self, collection, query, normalize : bool = True, top_k : int = 5, save_path : str = None):
        """
        vector_db의 자료를 기반으로 query와 관련 있는 문서 k개 필터링, 응답 생성
        (Naive RAG)
        - collection : 벡터 DB
        - query : 질의
        - normalize : 임베딩 정규화 여부
        - top_k : 검색 결과 상위 k개
        - save_path : 필터링한한 유사 자료들 저장할 곳
        """
        self.collection = collection
        self.query = query
        lawembeddings = LawEmbeddings()
        self.embedding_fn = lawembeddings.create_query_embedding # 이미 normalize된 임베딩
        self.normalize = normalize
        self.top_k = top_k
        self.documents : List[Dict] = []
        self.embeddings : Optional[np.ndarray] = None
        self.save_path = save_path # 유사한 자료들 저장할 곳

    def normalize_emb(self, x : np.ndarray) -> np.ndarray:
        """유사도 계산을 위해 임베딩 정규화"""
        # 1차원인 경우
        if x.ndim == 1:
            denom = np.linalg.norm(x) + 1e-10
            return x / denom
        # 2차원인 경우
        else:
            denom = np.linalg.norm(x, axis = 1, keepdims = True) + 1e-10
            return x / denom
    
    def search(self, query_embedding : np.ndarray, k : int = 5, where : Optional[Dict] = None) -> List[Dict]:
        """기본 벡터 검색(Naive RAG)"""
        try:
            if self.normalize:
                query_embedding = self.normalize_emb(query_embedding)

            results = self.collection.query(
                query_embeddings = [query_embedding.tolist()],
                n_results = k,
                where = where,
                include = ['documents', 'metadatas', 'distances']
            )

            formatted_results = []
            for i in range(len(results['documents'][0])):
                result = {
                    "index" : i, # 검색 결과 순서 기반
                    "text" : results['documents'][0][i],
                    "metadata" : results['metadatas'][0][i],
                    "relevance_score" : 1 - results["distances"][0][i],
                    "search_rank" : i # 벡터 검색 순위
                }
                formatted_results.append(result)
            return formatted_results
        
        except Exception as e:
            print(f"검색 중 오류 발생 : {e}")
            return []


    def save_filtered(self):
        """유사한 자료들을 모아 저장해둠"""
        self.save_path.parent.mkdir(parents = True, exist_ok = True)
        with open(self.save_path, "wb") as f:
            pickle.dump({
                "documents" : self.documents,
                "embeddings" : self.embeddings.astype(np.float32),
                "normalize" : self.normalize
            }, f)
            
    def load_filtered(self) -> True:
        if not self.save_path.exists():
            return False
        with open(self.save_path, "rb") as f:
            data = pickle.load(f)
        self.documents = data.get("documents", [])
        self.embeddings = data.get("embeddings", None)
        self.normalize = data.get("normalize", True)
        return True

In [ ]:
import chromadb
from pathlib import Path
import numpy as np

# 1. ChromaDB 버전 확인
print(f"ChromaDB 버전: {chromadb.__version__}")

# 2. 데이터베이스 상태 확인
project_root = Path("C:/Users/SAMSUNG/Desktop/Grad_School/RAG_LAW")
lawdb_path = project_root / "LawDB"

client = chromadb.PersistentClient(path=str(lawdb_path))
collection = client.get_or_create_collection("laws")

# 3. 컬렉션 정보 확인
print(f"문서 수: {collection.count()}")
print(f"컬렉션 메타데이터: {collection.metadata}")

# 4. 첫 번째 문서의 임베딩 확인
try:
    # 모든 문서 가져오기
    all_docs = collection.get(include=['embeddings'])
    if all_docs['embeddings']:
        first_emb = np.array(all_docs['embeddings'][0])
        print(f"첫 번째 임베딩 shape: {first_emb.shape}")
        print(f"첫 번째 임베딩 norm: {np.linalg.norm(first_emb)}")
        
        # 정규화 여부 확인
        if abs(np.linalg.norm(first_emb) - 1.0) < 0.01:
            print("✅ 임베딩이 정규화되어 있음")
        else:
            print("❌ 임베딩이 정규화되지 않음")
    else:
        print("❌ 임베딩이 없음")
        
except Exception as e:
    print(f"임베딩 확인 중 오류: {e}")

# 5. 간단한 검색 테스트
try:
    # 더미 임베딩으로 검색 테스트
    dummy_emb = np.random.rand(1024).astype(np.float32)
    dummy_emb = dummy_emb / np.linalg.norm(dummy_emb)  # 정규화
    
    results = collection.query(
        query_embeddings=[dummy_emb.tolist()],
        n_results=1,
        include=['documents', 'metadatas', 'distances']
    )
    print("✅ 검색 테스트 성공")
    
except Exception as e:
    print(f"❌ 검색 테스트 실패: {e}")

ChromaDB 버전: 1.0.13


InternalError: Database error: error returned from database: (code: 1) no such table: acquire_write

In [ ]:
import os
os.chdir('C:/Users/SAMSUNG/Desktop/Grad_School/RAG_LAW')
embeddings = np.load('DATA/laws_embedded.npy')
print(f"임베딩 shape: {embeddings.shape}")
print(f"임베딩 dtype: {embeddings.dtype}")
print(f"첫 번째 임베딩 norm: {np.linalg.norm(embeddings[0])}")
print(f"임베딩 범위: {embeddings.min():.4f} ~ {embeddings.max():.4f}")

# 2. BGE-M3 모델로 생성된 임베딩인지 확인
if abs(np.linalg.norm(embeddings[0]) - 1.0) < 0.01:
    print("✅ 임베딩이 이미 정규화되어 있음")
else:
    print("❌ 임베딩이 정규화되지 않음")

임베딩 shape: (19627, 1024)
임베딩 dtype: float32
첫 번째 임베딩 norm: 1.0000001192092896
임베딩 범위: -0.3048 ~ 0.4060
✅ 임베딩이 이미 정규화되어 있음


In [ ]:
# project_root = Path(__file__).resolve().parent.parent
project_root = Path("C:/Users/SAMSUNG/Desktop/Grad_School/RAG_LAW")
vector_db_path = project_root / "LawDB"

client = chromadb.PersistentClient(path = str(vector_db_path))
collection = client.get_or_create_collection("laws")

query = "개인정보를 처리하는 기관은 어디인가요?"

search_engine = NaiveSearchEngine(collection, query)
query_embedding = search_engine.embedding_fn(query) 

results = search_engine.search(query_embedding)
for r in results:
    print(r["relevance_score"], r["metadata"].get("law_name"), r["text"][:80], "...")

검색 중 오류 발생 : Error executing plan: Error sending backfill request to compactor: Error constructing hnsw segment reader: Error creating hnsw segment reader: Error loading hnsw index
